In [100]:
import numpy as np
import pandas as pd
import model
import nessra_genes_sets_generator as ng
import mygene 
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Import `StandardScaler` from `sklearn.preprocessing`
from sklearn.preprocessing import StandardScaler
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB


# Matrix and Objective vector preparation

In [108]:
#------------- MATRIX and CLASS VECTOR
matrix = pd.read_csv("~/Desktop/expressionMatrix5.tsv", sep = "\t", header=0)
#class_vector = pd.read_csv("~/GoogleDrive/DataMiningLab/Model/objective.tsv", sep = "\t", header=0)
print("Matrix shape: {}".format(matrix.shape))
#print(matrix.head(2))
print("")

Matrix shape: (60482, 103)



In [109]:
# transposing matrix
matrix_t = matrix.T
matrix_t.columns = matrix_t.iloc[0]
matrix_t = matrix_t.drop("Gene_ID", axis=0)

# Resetting column names without the version of ENS
col_names = list(matrix_t.columns.values)
new_col_names = []
for name in col_names:
    temp_split = name.split(".")
    new_col_names.append(temp_split[0])
matrix_t.columns = new_col_names
print("Transpose matrix shape: {}".format(matrix_t.shape))
print("")

Transpose matrix shape: (102, 60482)



In [110]:
# class vector 
class_vector = pd.read_csv("~/Desktop/objective.tsv", sep = "\t", header=0)
print("Objective shape: {}".format(class_vector.shape))  

print(len([x for x in class_vector["over2years"] if x <1])/len(class_vector["over2years"]))
print(len(class_vector["over2years"]))

# generating barcode of samples
samples_bar = []
for name in list(matrix_t.index.values):
    samples_bar.append(name[:-4])
fil_for_class = class_vector.barcode.isin(samples_bar)
class_vector = class_vector[fil_for_class]
print("Objective shape after filter for matrix samples: {}".format(class_vector.shape))

# Final vector for classifying
train = class_vector["over2years"]
print("Final vector for classiftibg is a {}".format(type(train)))
print("")

Objective shape: (104, 3)
0.6634615384615384
104
Objective shape after filter for matrix samples: (102, 3)
Final vector for classiftibg is a <class 'pandas.core.series.Series'>



# NESSRA

importing and preparing output genes

In [5]:
#----------------- NESSRA -----------------
print("----SVM on NESSRA----")
print("")
# importing files output
list_apc = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160679_Hs.expansion', sep = ",", header=1),
            pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160679_Hs.interactions', sep = ",", header=1)]
list_smad4 = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160680_Hs.expansion', sep = ",", header=1),
            pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160680_Hs.interactions', sep = ",", header=1)]
list_kras = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/160812_Hs.expansion', sep = ",", header=1),
            [None]]
list_braf = [pd.read_csv('~/GoogleDrive/DataMiningLab/Nessra/147717_Hs.expansion', sep = ",", header=1),
            [None]]

# importing tcode conversion matrix
tcode_mat = pd.read_csv('~/GoogleDrive/DataMiningLab/hgnc_filtered_anno.csv', sep = ",", header=0)

----SVM on NESSRA----



/home/riccardo/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Set filter tresholds and max genes per set here!
tcode_symbol_dict = ng.tcodeSymbolDictGen(tcode_mat)

I'm creating tcode-to-Symbol dictionary, this will take a minute
Done, dictionary ready


In [124]:
# Constructing sets
freq_treshold = 0.3
n_genes_per_file = 160


set_apc = ng.setGen(list_apc[0], list_apc[1],freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=True)
set_smad4 = ng.setGen(list_smad4[0], list_smad4[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
set_kras = ng.setGen(list_kras[0], list_kras[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
set_braf = ng.setGen(list_braf[0], list_braf[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)

list_of_sets = [set_apc, set_smad4, set_braf, set_braf ]
print("Len of set apc: {}".format(len(set_apc)))
print("Len of set smad4: {}".format(len(set_smad4)))
print("Len of set kras: {}".format(len(set_kras)))
print("Len of set braf: {}".format(len(set_braf)))

querying 1-160...done.
Finished.
5 input query terms found dup hits:
	[('NKAPP1', 2), ('LINC00237', 2), ('MIR4500HG', 2), ('AQP4-AS1', 2), ('MIR99AHG', 2)]
7 input query terms found no hit:
	['FAM19A1', 'C8orf46', 'KIAA1468', 'TROVE2', 'FAM57B', 'KIAA1107', 'CTAGE5']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-160...done.
Finished.
5 input query terms found dup hits:
	[('SDAD1P1', 2), ('MIR4523', 2), ('CHCHD2P6', 2), ('MIR99AHG', 2), ('HCG25', 8)]
2 input query terms found no hit:
	['LINC00441', 'PROSC']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-160...done.
Finished.
4 input query terms found dup hits:
	[('LY6G6E', 8), ('MIR21', 2), ('HNF1A-AS1', 2), ('GLIS3-AS1', 2)]
3 input query terms found no hit:
	['KIAA0430', 'ISPD', 'ALS2CR11']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-160...done.
Finished.
1 input query terms found dup hits

# data splitting

splitting dataset, training model and evaluation

In [126]:
#------------ SVM ANALYSIS ------------
ns_mat = model.select_table(matrix_t, list_of_sets)
print("Transpose matrix filtered shape: {}".format(ns_mat.shape))

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(ns_mat.values, train.values, test_size=0.3,random_state=100) # 80% training and 20% test


# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

# SVM

In [129]:
# train the model
clf_ns = model.training(X_train, y_train)

y_pred = clf_ns.predict(X_test)
print("")
print("--- Evaluating test set ---")
print("Test matrix shape: {}  \nTest class vector: {} \n".format(X_test.shape, len(y_test)))
print("Frequency filter used in NESSRA output: {}, final number of genes: {}". format(freq_treshold, X_test.shape[1]))
print("")
# Model Accuracy: how often is the classifier correct?
print("AUC:",metrics.roc_auc_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))
print("")
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in y_pred if x <1])/len(y_pred)))


{'C': 10.0, 'gamma': 1.0}
fit_time 0.002627293268839518
score_time 0.006701946258544922
test_accuracy 0.6630434782608695
test_f1 0.0
test_precision 0.0
test_recall 0.0

--- Evaluating test set ---
Test matrix shape: (31, 455)  
Test class vector: 31 

Frequency filter used in NESSRA output: 0.3, final number of genes: 455

Accuracy: 0.6451612903225806
Precision: 0.0
Recall: 0.0

Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 1.0


/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site

# Naive Bayes Model

In [130]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
g_model = GaussianNB()

# Train the model using the training sets
g_model.fit(X_train,y_train)

#Predict Output
predicted= g_model.predict(X_test)

print("Nessra treshold of {}, selected max {} genes per set:".format(freq_treshold, n_genes_per_file))
print("")
print("Len of ens id set apc: {}".format(len(set_apc)))
print("Len of ens id set smad4: {}".format(len(set_smad4)))
print("Len of ens id set kras: {}".format(len(set_kras)))
print("Len of ens id set braf: {}".format(len(set_braf)))
print("")
print("Transpose matrix intersection with sets shape: {}".format(ns_mat.shape))
print("")
print("---- Naive Bayes ----")
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in predicted if x <1])/len(predicted)))
print("")
print(predicted)
print(y_test)
print("AUC:",metrics.roc_auc_score(y_test, predicted))

Nessra treshold of 0.3, selected max 160 genes per set:

Len of ens id set apc: 162
Len of ens id set smad4: 189
Len of ens id set kras: 169
Len of ens id set braf: 175

Transpose matrix intersection with sets shape: (102, 455)

---- Naive Bayes ----
Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 0.45161290322580644

[1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 0]
[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
Accuracy: 0.6774193548387096


# Using single NESSRA FILES

In [253]:
# Constructing sets
freq_treshold = 0.3
n_genes_per_file = 250


#set_apc = ng.setGen(list_apc[0], list_apc[1],freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=True)
set_smad4 = ng.setGen(list_smad4[0], list_smad4[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
#set_kras = ng.setGen(list_kras[0], list_kras[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
set_braf = ng.setGen(list_braf[0], list_braf[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)

#list_of_sets = [set_apc, set_smad4, set_braf, set_braf ]
list_of_sets = [set_smad4, set_braf]
#print("Len of set apc: {}".format(len(set_apc)))
print("Len of set smad4: {}".format(len(set_smad4)))
#print("Len of set kras: {}".format(len(set_kras)))
print("Len of set braf: {}".format(len(set_braf)))

querying 1-250...done.
Finished.
10 input query terms found dup hits:
	[('SDAD1P1', 2), ('MIR4523', 2), ('CHCHD2P6', 2), ('MIR99AHG', 2), ('HCG25', 8), ('URB1-AS1', 2), ('
3 input query terms found no hit:
	['LINC00441', 'PROSC', 'ZCCHC11']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-250...done.
Finished.
4 input query terms found dup hits:
	[('HCG18', 8), ('MIR4727', 2), ('PCBP1-AS1', 2), ('POLR2J4', 2)]
7 input query terms found no hit:
	['CECR5', 'MARCH6', 'SSFA2', 'FAM49A', '2019-08-03 00:00:00', 'DIRC2', '2019-08-02 00:00:00']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
Len of set smad4: 279
Len of set braf: 275


In [254]:
ns_mat = model.select_table(matrix_t, list_of_sets)
print("Transpose matrix filtered shape: {}".format(ns_mat.shape))
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(ns_mat.values, train.values, test_size=0.3,random_state=100) # 70% training and 30% test
# Define the scaler 
scaler = StandardScaler().fit(X_train)
# Scale the train set
X_train = scaler.transform(X_train)
# Scale the test set
X_test = scaler.transform(X_test)

Transpose matrix filtered shape: (102, 482)


### SVM

In [255]:
# train the model
clf_ns = model.training(X_train, y_train)

y_pred = clf_ns.predict(X_test)
print("")
print("Nessra treshold of {}, selected max {} genes per set:".format(freq_treshold, n_genes_per_file))
print("")
#print("Len of ens id set apc: {}".format(len(set_apc)))
print("Len of ens id set smad4: {}".format(len(set_smad4)))
#print("Len of ens id set kras: {}".format(len(set_kras)))
print("Len of ens id set braf: {}".format(len(set_braf)))
print("")
print("Transpose matrix intersection with sets shape: {}".format(ns_mat.shape))
print("")
print("---- SVM ----")
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in y_pred if x <1])/len(y_pred)))
print("")
print(y_test)
print(y_pred)
print("AUC:",metrics.roc_auc_score(y_test, y_pred))

{'C': 10.0, 'gamma': 1.0}
fit_time 0.002952019373575846
score_time 0.008234262466430664
test_accuracy 0.6612318840579711
test_f1 0.0
test_precision 0.0
test_recall 0.0

Nessra treshold of 0.3, selected max 250 genes per set:

Len of ens id set smad4: 279
Len of ens id set braf: 275

Transpose matrix intersection with sets shape: (102, 482)

---- Naive Bayes ----
Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 1.0

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
AUC: 0.5


/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/riccardo/miniconda3/lib/python3.7/site

### Bayes

In [256]:
#Create a Gaussian Classifier
g_model = GaussianNB()

# Train the model using the training sets
g_model.fit(X_train,y_train)
predicted= g_model.predict(X_test)

print("Nessra treshold of {}, selected max {} genes per set:".format(freq_treshold, n_genes_per_file))
print("")
#print("Len of ens id set apc: {}".format(len(set_apc)))
print("Len of ens id set smad4: {}".format(len(set_smad4)))
#print("Len of ens id set kras: {}".format(len(set_kras)))
print("Len of ens id set braf: {}".format(len(set_braf)))
print("")
print("Transpose matrix intersection with sets shape: {}".format(ns_mat.shape))
print("")
print("---- Naive Bayes ----")
print("Proportion of 0s in test y: {}".format(len([x for x in y_test if x <1])/len(y_test)))
print("Proportion of 0s in prediction y: {}".format(len([x for x in predicted if x <1])/len(predicted)))
print("")
print(y_test)
print(predicted)
print("AUC:",metrics.roc_auc_score(y_test, predicted))


Nessra treshold of 0.3, selected max 250 genes per set:

Len of ens id set smad4: 279
Len of ens id set braf: 275

Transpose matrix intersection with sets shape: (102, 482)

---- Naive Bayes ----
Proportion of 0s in test y: 0.6451612903225806
Proportion of 0s in prediction y: 0.45161290322580644

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 1 0 0 1 1 0]
AUC: 0.7795454545454545


In [257]:
def acc_GenesReturn(list_smad4, list_braf, freq_treshold, n_genes_per_file, tcode_symbol_dict, matrix_t, returnLists=False, verbose=False):
    
    #set_apc = ng.setGen(list_apc[0], list_apc[1],freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=True)
    set_smad4 = ng.setGen(list_smad4[0], list_smad4[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
    #set_kras = ng.setGen(list_kras[0], list_kras[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)
    set_braf = ng.setGen(list_braf[0], list_braf[1], freq_treshold, n_genes_per_file, tcode_symbol_dict, useIntFile=False)

    #list_of_sets = [set_apc, set_smad4, set_braf, set_braf ]
    list_of_sets = [set_smad4, set_braf]
    ns_mat = model.select_table(matrix_t, list_of_sets)
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(ns_mat.values, train.values, test_size=0.3,random_state=100) # 70% training and 30% test
    # Define the scaler 
    scaler = StandardScaler().fit(X_train)
    # Scale the train set
    X_train = scaler.transform(X_train)
    # Scale the test set
    X_test = scaler.transform(X_test)

    #Create a Gaussian Classifier
    g_model = GaussianNB()
    # Train the model using the training sets
    g_model.fit(X_train,y_train)
    predicted= g_model.predict(X_test)

    acc = metrics.roc_auc_score(y_test, predicted)
    
    ens_gene_list = None
    symbol_gene_list = None
    
    if returnLists:
        ens_gene_list = list(ns_mat.columns.values)
        mg = mygene.MyGeneInfo()
        symbol_genes = mg.querymany(ens_gene_list, scopes='ensembl.gene', fields='symbol', species='human')
        symbol_gene_list = []
        for el in symbol_genes:
            if "symbol" in el:
                symbol_gene_list.append(el["symbol"])
        if verbose:
            print("len of ens genes: {}, len of symbol genes: {}".format(len(ens_gene_list), len(gene_list)))
            print(', '.join(symbol_gene_list))
    
    return (n_genes_per_file, acc, ens_gene_list, symbol_gene_list)

def BayesModelOut(ns_mat, train):
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(ns_mat.values, train.values, test_size=0.3,random_state=100) # 70% training and 30% test

    # Define the scaler 
    scaler = StandardScaler().fit(X_train)
    # Scale the train set
    X_train = scaler.transform(X_train)
    # Scale the test set
    X_test = scaler.transform(X_test)

    #Create a Gaussian Classifier
    g_model = GaussianNB()

    # Train the model using the training sets
    g_model.fit(X_train,y_train)

    predicted= g_model.predict(X_test)

    #print("Nessra treshold of {}, selected max {} genes per set:".format(freq_treshold, n_genes_per_file))
    #print("")
    #print("Len of ens id set apc: {}".format(len(set_apc)))
   #print("Len of ens id set smad4: {}".format(len(set_smad4)))
    #print("Len of ens id set kras: {}".format(len(set_kras)))
    #print("Len of ens id set braf: {}".format(len(set_braf)))
    #print("")
    #print("Transpose matrix intersection with sets shape: {}".format(ns_mat.shape))
    #print("")
    print("---- Naive Bayes ----")
    print("Proportion of 0s in test y: {}".format(round(len([x for x in y_test if x <1])/len(y_test), 4)))
    print("Proportion of 0s in prediction y: {}".format(round(len([x for x in predicted if x <1])/len(predicted),4)))
    print("")
    print(y_test)
    print(predicted)
    print("AUC:",round(metrics.roc_auc_score(y_test, predicted), 4))

In [258]:
freq_treshold = 0.3

acc_results = []
for n_genes in range(1,21):
    n_genes_per_file = n_genes*10
    tmp = acc_GenesReturn(list_smad4, list_braf, freq_treshold, n_genes_per_file, tcode_symbol_dict, matrix_t, returnLists=True)
    acc_results.append(tmp)

querying 1-10...done.
Finished.
querying 1-10...done.
Finished.
querying 1-20...done.
Finished.
querying 1-20...done.
Finished.
1 input query terms found dup hits:
	[('SDAD1P1', 2)]
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-20...done.
Finished.
querying 1-40...done.
Finished.
querying 1-30...done.
Finished.
1 input query terms found dup hits:
	[('SDAD1P1', 2)]
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-30...done.
Finished.
1 input query terms found dup hits:
	[('HCG18', 8)]
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-60...done.
Finished.
querying 1-40...done.
Finished.
1 input query terms found dup hits:
	[('SDAD1P1', 2)]
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-40...done.
Finished.
1 input query terms found dup hits:
	[('HCG18', 8)]
Pass "returnall=True" to return complete lists

querying 1-180...done.
Finished.
1 input query terms found dup hits:
	[('HCG18', 8)]
7 input query terms found no hit:
	['CECR5', 'MARCH6', 'SSFA2', 'FAM49A', '2019-08-03 00:00:00', 'DIRC2', '2019-08-02 00:00:00']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-345...done.
Finished.
querying 1-190...done.
Finished.
7 input query terms found dup hits:
	[('SDAD1P1', 2), ('MIR4523', 2), ('CHCHD2P6', 2), ('MIR99AHG', 2), ('HCG25', 8), ('URB1-AS1', 2), ('
2 input query terms found no hit:
	['LINC00441', 'PROSC']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-190...done.
Finished.
1 input query terms found dup hits:
	[('HCG18', 8)]
7 input query terms found no hit:
	['CECR5', 'MARCH6', 'SSFA2', 'FAM49A', '2019-08-03 00:00:00', 'DIRC2', '2019-08-02 00:00:00']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-365...done.
Finished.
querying 1-200...done.
Fi

In [259]:
print("genes per set used ---- auc")
for i in acc_results:
    print("                {} ---- {}".format(i[0], round(i[1], 4)))
print(len(acc_results))

genes per set used ---- auc
                10 ---- 0.6432
                20 ---- 0.7091
                30 ---- 0.7841
                40 ---- 0.8045
                50 ---- 0.7545
                60 ---- 0.8045
                70 ---- 0.85
                80 ---- 0.7591
                90 ---- 0.7386
                100 ---- 0.7841
                110 ---- 0.8295
                120 ---- 0.8045
                130 ---- 0.8045
                140 ---- 0.7841
                150 ---- 0.7841
                160 ---- 0.8091
                170 ---- 0.8091
                180 ---- 0.8091
                190 ---- 0.8091
                200 ---- 0.8091
20


In [274]:
genes_10_40 = []
for gene in acc_results[3][2]:
    if gene not in acc_results[0][2]:
        genes_10_40.append(gene)
print("len of genes_10_40: {}".format(len(genes_10_40)))


genes_60_70 = []
for gene in acc_results[6][2]:
    if gene not in acc_results[5][2]:
        genes_60_70.append(gene)
print("len of genes_60_70: {}".format(len(genes_60_70)))

genes_90_110 = []
for gene in acc_results[10][2]:
    if gene not in acc_results[8][2]:
         genes_90_110.append(gene)
print("len of genes_90_110: {}".format(len(genes_90_110)))

genes_150_160 = []
for gene in acc_results[15][2]:
    if gene not in acc_results[14][2]:
        genes_150_160.append(gene)
print("len of genes_150_160: {}".format(len(genes_150_160)))


len of genes_10_40: 60
len of genes_60_70: 17
len of genes_90_110: 39
len of genes_150_160: 17


In [275]:
print("!!!!!!!!!!! 1 SET !!!!!!!!!!!")
print("")
print("---------------------------------------")
print("Using set/s genes_10_40:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_10_40)])
BayesModelOut(ns_mat, train)
print("")


print("---------------------------------------")
print("Using set/s genes_60_70:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_60_70)])
BayesModelOut(ns_mat, train)
print("")


print("---------------------------------------")
print("Using set/s genes_90_110:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_90_110)])
BayesModelOut(ns_mat, train)
print("")

print("---------------------------------------")
print("Using set/s genes_150_160:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_150_160)])
BayesModelOut(ns_mat, train)
print("")

print("!!!!!!!!!!! COMBINATIONS WITH 2 SETS !!!!!!!!!!!")
print("")


print("---------------------------------------")
print("Using set/s genes_10_40, genes_60_70:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_10_40), set(genes_60_70)])
BayesModelOut(ns_mat, train)
print("")

print("---------------------------------------")
print("Using set/s genes_10_40, genes_90_110:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_10_40), set(genes_90_110)])
BayesModelOut(ns_mat, train)
print("")


print("---------------------------------------")
print("Using set/s genes_10_40, genes_150_160:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_10_40), set(genes_150_160)])
BayesModelOut(ns_mat, train)
print("")


print("---------------------------------------")
print("Using set/s genes_60_70, genes_90_110:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_60_70), set(genes_90_110)])
BayesModelOut(ns_mat, train)
print("")

print("---------------------------------------")
print("Using set/s genes_60_70, genes_150_160:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_60_70), set(genes_150_160)])
BayesModelOut(ns_mat, train)
print("")

print("---------------------------------------")
print("Using set/s genes_90_110, genes_150_160:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_90_110), set(genes_150_160)])
BayesModelOut(ns_mat, train)
print("")

print("!!!!!!!!!!! COMBINATIONS WITH 3 SETS !!!!!!!!!!!")
print("")

print("---------------------------------------")
print("Using set/s genes_10_40, genes_60_70, genes_90_110:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_10_40), set(genes_60_70), set(genes_90_110)])
BayesModelOut(ns_mat, train)
print("")

print("---------------------------------------")
print("Using set/s genes_10_40, genes_60_70, genes_150_160:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_10_40), set(genes_60_70), set(genes_150_160)])
BayesModelOut(ns_mat, train)
print("")

print("---------------------------------------")
print("Using set/s genes_60_70, genes_90_110, genes_150_160:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_60_70), set(genes_90_110), set(genes_150_160)])
BayesModelOut(ns_mat, train)
print("")

print("!!!!!!!!!!! 4 SETS !!!!!!!!!!!")

print("---------------------------------------")
print("Using set/s genes_10_40, genes_60_70, genes_90_110, genes_150_160:")
print("")
ns_mat = model.select_table(matrix_t, [set(genes_10_40), set(genes_60_70), set(genes_90_110), set(genes_150_160)])
BayesModelOut(ns_mat, train)
print("")


!!!!!!!!!!! 1 SET !!!!!!!!!!!

---------------------------------------
Using set/s genes_10_40:

---- Naive Bayes ----
Proportion of 0s in test y: 0.6452
Proportion of 0s in prediction y: 0.3871

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[1 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 1 1 1 0]
AUC: 0.6591

---------------------------------------
Using set/s genes_60_70:

---- Naive Bayes ----
Proportion of 0s in test y: 0.6452
Proportion of 0s in prediction y: 0.6452

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[0 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
AUC: 0.7182

---------------------------------------
Using set/s genes_90_110:

---- Naive Bayes ----
Proportion of 0s in test y: 0.6452
Proportion of 0s in prediction y: 0.4516

[0 0 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0]
[1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 1 1 1 1 1]
AUC: 0.6386

---------------------------------------
Using set/

In [271]:
new_genes_set = set(genes_10_40).union(set(genes_60_70))
new_genes_set = new_genes_set.union(set(genes_90_110))
new_genes_set = new_genes_set.union(set(genes_150_160))
print(len(new_genes_set))

mg = mygene.MyGeneInfo()
symbol_genes = mg.querymany(list(new_genes_set), scopes='ensembl.gene', fields='symbol', species='human')
symbol_gene_list = []
for el in symbol_genes:
    if "symbol" in el:
        symbol_gene_list.append(el["symbol"])
print("len of ens genes: {}, len of symbol genes: {}".format(len(new_genes_set), len(symbol_gene_list)))
print(', '.join(symbol_gene_list))

133
querying 1-133...done.
Finished.
len of ens genes: 133, len of symbol genes: 133
SMAD2, STAT5B, CPEB2, CHKB, NUP98, AHCTF1, AIG1, SHF, HINT2, BID, PAM, APLF, WASHC3, ZBTB17, UNKL, NFKBID, RICTOR, RSPO1, NOP16, SDAD1P1, PHF1, MED12, RFX2, SLMAP, ZNF501, TMEM19, ZNF266, CSAD, PELI2, IMP4, LMTK2, UIMC1, PTK2B, TRMT10A, SLCO4A1, IRGQ, PRKCB, DOT1L, TTC39C, TMEM128, SECISBP2, MTIF3, PTPN4, FBXO34, SSH2, ZNF420, MIS12, ANAPC16, KLF3, ZNF140, ADM2, RAB28, ZKSCAN7, KDM6A, SLC45A4, C2orf42, ZFC3H1, FAM76B, KAT6A, IFT57, ARF4, SIK3, JMJD1C, CRTC2, PTEN, MAPK1, RNF103, DDX41, GABPA, ZMYM2, FNIP1, RANBP2, HMGB1, FBXO11, TBCEL, PLGRKT, ACBD4, PELI1, CREBRF, KCTD6, GSTP1, NR2C2, LPGAT1, TOR3A, JMJD6, PPP1R12A, ARFIP1, PSME4, USP38, ZNF131, DXO, PLSCR1, UBXN7, SRPK2, RLIM, HDHD2, HCG18, SMIM19, TNKS, BANP, GOLGA2, ADNP2, DENND1A, PPIL3, KCNAB2, KIAA0232, ADK, TMCO4, ZNF426, TTLL1, C11orf71, TMEM216, FBXL17, SCLT1, KDM5C, MBTD1, KANSL1, DHRS12, YTHDC1, U2AF1L4, RNF44, EXOSC4, WWC2, NFAT5, ANKRD11,